In [14]:
import numpy as np
import pandas as pd
from Bio import SeqIO

In [15]:
def read_fasta_file(fasta_file):
    sequence = ""

    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence += str(record.seq)

    return sequence

In [16]:
def parse_gtf_file(gtf_file):
    exons = {}

    with open(gtf_file, 'r') as f:
        for line in f:
            if not line.startswith('#'):
                cols = line.strip().split('\t')
                feature_type = cols[2]
                if feature_type == 'exon':
                    chromosome = cols[0]
                    start = int(cols[3])
                    end = int(cols[4])
                    gene_id = cols[8].split(';')[0].split('"')[1]
                    if gene_id not in exons:
                        exons[gene_id] = []
                    exons[gene_id].append((start, end))

    return exons

In [17]:
fasta_0_Coli = './data/samples/sample_0/reference_genome_545778205.fasta'
fasta_0_Subtilis = './data/samples/sample_5/reference_genome_255767013.fasta'
fasta_1_Coli = './data/samples/sample_3/reference_genome_682117612.fasta'
fasta_1_Subtilis = './data/samples/sample_6/reference_genome_1678549200.fasta'

fasta_0_Coli = read_fasta_file(fasta_0_Coli)
fasta_0_Subtilis = read_fasta_file(fasta_0_Subtilis)
fasta_1_Coli = read_fasta_file(fasta_1_Coli)
fasta_1_Subtilis = read_fasta_file(fasta_1_Subtilis)

In [18]:
gtf_0_Coli = './data/samples/sample_0/545778205_gtf_merged.gtf'
gtf_0_Subtilis = './data/samples/sample_5/255767013_gtf_merged.gtf'
gtf_1_Coli = './data/samples/sample_3/682117612_gtf_merged.gtf'
gtf_1_Subtilis = './data/samples/sample_6/1678549200_gtf_merged.gtf'

gtf_0_Coli = parse_gtf_file(gtf_0_Coli)
gtf_0_Subtilis = parse_gtf_file(gtf_0_Subtilis)
gtf_1_Coli = parse_gtf_file(gtf_1_Coli)
gtf_1_Subtilis = parse_gtf_file(gtf_1_Subtilis)

In [19]:
n_pad = 300
n_annotate = 300
n_window = 900

dict_ = []

for sequence, exon in zip([fasta_0_Coli, fasta_1_Coli, fasta_0_Subtilis, fasta_1_Subtilis],
                           [gtf_0_Coli, gtf_1_Coli, gtf_0_Subtilis, gtf_1_Subtilis]):
    for j in range(0, len(sequence), 100):
        befor_start = j - n_pad
        annotate_start = j
        after_start = j + n_annotate + n_pad

        befor_end = j
        annotate_end = min(j + n_annotate, len(sequence))
        after_end = j + n_window

        window = []

        if befor_start < 0:
            window += ['N'] * abs(befor_start)
            window += sequence[0 : befor_end]
        else:
            window += sequence[befor_start : befor_end]

        window += sequence[annotate_start : annotate_end]

        if after_end > len(sequence):
            window += sequence[after_start : len(sequence)]
            window += ['N'] * abs(n_window - len(window))
        else:
            window += sequence[after_start : after_end]

        it = 0
        for k in range(annotate_start, annotate_end):
            it += 1 if any(start <= k < end for transcript_exons in exon.values() for start, end in transcript_exons) else 0

        label = 1 if it / n_annotate > 0.66 else 0

        iteration_dict = {
            'seq_overlap': ''.join(window),
            'seq_med': sequence[annotate_start : annotate_end],
            'label': label
        }

        dict_.append(iteration_dict)



In [15]:
df = pd.DataFrame(dict_)
df['label'].value_counts()

label
1    95587
0    81415
Name: count, dtype: int64

In [16]:
def fix_window(row):
    overlap_len = len(row['seq_overlap'])
    med_len = len(row['seq_med'])

    if overlap_len < 900:
        row['seq_overlap'] += 'N' * (900 - overlap_len)

    if med_len < 300:
        row['seq_med'] += 'N' * (300 - med_len)

    return row

df = df.apply(fix_window, axis = 1)

In [ ]:
def dismiss_N(row):
    if 'N' in row['seq_overlap'] or 'N' in row['seq_med']:
        return False
    else:
        return True
    

df = df[df.apply(dismiss_N, axis=1)]
df['label'].value_counts()

In [17]:
df_label_0 = df[df['label'] == 0]
df_label_1 = df[df['label'] == 1]

df_label_0.to_csv('df_label_0.csv', index = False)
df_label_1.to_csv('df_label_1.csv', index = False)

In [18]:
def write_fasta(df, label, filename_prefix):
    with open(f'{filename_prefix}_{label}.fasta', 'w') as f:
        for idx, row in df.iterrows():
            sequence = row['seq_med']
            f.write(f'>BA000007.3_{idx}\n')
            f.write(f'{sequence}\n')


write_fasta(df_label_0, 0, 'class_0')
write_fasta(df_label_1, 1, 'class_1')

In [ ]:
!python3 ../../Soft/MathFeature/preprocessing/preprocessing.py -i class_0_0.fasta -o class_0_0_preprocess.fasta
!python3 ../../Soft/MathFeature/preprocessing/preprocessing.py -i class_1_1.fasta -o class_1_1_preprocess.fasta

In [ ]:
!python3 ../../Soft/MathFeature/methods/ExtractionTechniques.py -i class_0_0_preprocess.fasta -o class_0_feature_1.csv -l DNA -t DNC -seq 1
!python3 ../../Soft/MathFeature/methods/ExtractionTechniques.py -i class_1_1_preprocess.fasta -o class_1_feature_1.csv -l DNA -t DNC -seq 1

In [ ]:
!python3 ../../Soft/MathFeature/methods/ExtractionTechniques.py -i class_0_0_preprocess.fasta -o class_0_feature_2.csv -l DNA -t TNC -seq 1
!python3 ../../Soft/MathFeature/methods/ExtractionTechniques.py -i class_1_1_preprocess.fasta -o class_1_feature_2.csv -l DNA -t TNC -seq 1

In [ ]:
!python3 ../../Soft/MathFeature/methods/FourierClass.py -i class_0_0_preprocess.fasta -o class_0_feature_3.csv -l mRNA -r 3
!python3 ../../Soft/MathFeature/methods/FourierClass.py -i class_1_1_preprocess.fasta -o class_1_feature_3.csv -l mRNA -r 3

In [ ]:
!python3 ../../Soft/MathFeature/methods/EntropyClass.py -i class_0_0_preprocess.fasta -o class_0_feature_4.csv -l mRNA -k 10 -e Shannon
!python3 ../../Soft/MathFeature/methods/EntropyClass.py -i class_1_1_preprocess.fasta -o class_1_feature_4.csv -l mRNA -k 10 -e Shannon

In [ ]:
!python3 ../../Soft/MathFeature/methods/AccumulatedNucleotideFrequency.py -n 1 -o class_0_feature_5.csv -r 2

In [ ]:
!python3 ../../Soft/MathFeature/methods/AccumulatedNucleotideFrequency.py -n 1 -o class_1_feature_5.csv -r 2

In [ ]:
!python3 ../../Soft/MathFeature/methods/CodingClass.py -i class_0_0_preprocess.fasta -o class_0_feature_6.csv -l lncRNA
!python3 ../../Soft/MathFeature/methods/CodingClass.py -i class_1_1_preprocess.fasta -o class_1_feature_6.csv -l lncRNA

In [ ]:
!python3 ../../Soft/MathFeature/methods/FickettScore.py -i class_0_0_preprocess.fasta -o class_0_feature_7.csv -l lncRNA -seq 1
!python3 ../../Soft/MathFeature/methods/FickettScore.py -i class_1_1_preprocess.fasta -o class_1_feature_7.csv -l lncRNA -seq 1

In [8]:
for i in range(1, 8): 
    for j in ['class_0_feature_', 'class_1_feature_']:
        df_temp = pd.read_csv(open(j + str(i) + '.csv', 'rb'))
        df_temp = df_temp.drop_duplicates(keep = 'first')
        df_temp.to_csv(j + str(i) + '.csv', index = False)

In [12]:
df_0 = pd.read_csv(open('df_label_0.csv', 'rb'))
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81399 entries, 0 to 81398
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   seq_overlap  81399 non-null  object
 1   seq_med      81399 non-null  object
 2   label        81399 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [13]:
df_add = pd.read_csv(open('class_0_feature_1.csv', 'rb'))
df_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81406 entries, 0 to 81405
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   nameseq  81406 non-null  object 
 1   AA       81406 non-null  float64
 2   AC       81406 non-null  float64
 3   AG       81406 non-null  float64
 4   AT       81406 non-null  float64
 5   CA       81406 non-null  float64
 6   CC       81406 non-null  float64
 7   CG       81406 non-null  float64
 8   CT       81406 non-null  float64
 9   GA       81406 non-null  float64
 10  GC       81406 non-null  float64
 11  GG       81406 non-null  float64
 12  GT       81406 non-null  float64
 13  TA       81406 non-null  float64
 14  TC       81406 non-null  float64
 15  TG       81406 non-null  float64
 16  TT       81406 non-null  float64
 17  label    81406 non-null  object 
dtypes: float64(16), object(2)
memory usage: 11.2+ MB
